<a href="https://colab.research.google.com/github/kunalgupta10/TextSummarizer-TestDataInAccurate-/blob/main/TextSummarizerProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Aug 14 08:54:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   74C    P0             43W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.55.2
Uninstalling transformers-4.55.2:
  Successfully uninstalled transformers-4.55.2
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
  Using cached accelerate-1.10.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.55.2-py3-none-any.whl (11.3 MB)
Using cached accelerate-1.10.0-py3-none-any.whl (374 kB)


In [4]:
!pip install -U transformers --quiet

In [5]:
!pip install evaluate

In [6]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset
from evaluate import load

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [9]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
dataset_samsum = load_dataset("knkarthick/samsum")

In [12]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [13]:
dataset_samsum["train"]["dialogue"][1]


'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [14]:
dataset_samsum["train"][1]["summary"]

'Olivia and Olivier are voting for liberals in this election. '

In [15]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 818, 819]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [16]:
def preprocess_function(batch):
    dialogues = [str(x) for x in batch["dialogue"]]
    summaries = [str(x) for x in batch["summary"]]

    # Tokenize the dialogues
    model_inputs = tokenizer(
        dialogues,
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

    # Tokenize the summaries (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            summaries,
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
dataset_samsum_pt = dataset_samsum.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset_samsum["train"].column_names
)

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
dataset_samsum_pt["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [19]:
from transformers import DataCollatorForSeq2Seq,Seq2SeqTrainingArguments,Seq2SeqTrainer

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

In [21]:
import evaluate

In [22]:
rouge = evaluate.load("rouge")

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels (ignored tokens) before decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects newline separated sentences for each summary
    decoded_preds = ["\n".join(pred.strip().split(". ")) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split(". ")) for label in decoded_labels]

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # Return results as percentages
    return {k: round(v * 100, 2) for k, v in result.items()}


In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./pegasus-samsum",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,  # Increase later
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=100
)

In [25]:
trainer = Seq2SeqTrainer(
    model=model_pegasus,
    args=training_args,
    train_dataset=dataset_samsum_pt["test"],
    eval_dataset=dataset_samsum_pt["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-1488543101.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [27]:
trainer.train()

wandb: Currently logged in as: gogogoyes785 (gogogoyes785-tata-consultancy-services) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,9.534100
200,8.812000
300,8.595800
400,8.376900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=410, training_loss=8.815876137338034, metrics={'train_runtime': 377.1069, 'train_samples_per_second': 2.172, 'train_steps_per_second': 1.087, 'total_flos': 2366471355236352.0, 'train_loss': 8.815876137338034, 'epoch': 1.0})

In [28]:
trainer.save_model("./pegasus-samsum")


In [29]:
test_text = dataset_samsum["test"][0]["dialogue"]
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding="max_length", max_length=1024)

In [31]:
if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    model_pegasus.to("cuda")

In [33]:
summary_ids = model_pegasus.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,
    num_beams=4,
    length_penalty=2.0,
    early_stopping=True
)

In [34]:
print("\nOriginal Dialogue:\n", test_text)
print("\nGenerated Summary:\n", tokenizer.decode(summary_ids[0], skip_special_tokens=True))
print("\nReference Summary:\n", dataset_samsum["test"][0]["summary"])


Original Dialogue:
 Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Generated Summary:
 Amanda is looking for Betty's number, but Hannah can't find it.<n>Hannah would rather she text Betty, because she'd rather she text him.

Reference Summary:
 Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
